# Imports and Config

In [ ]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/NLP\ Stock\ Prediction/New\ Work
!pip install flair
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import spacy
import random
from scipy.stats import zscore
import os
import string
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import regex as re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')
nltk.download('punkt')
nltk.download('wordnet')

/content/drive/My Drive/Colab Notebooks/NLP Stock Prediction/New Work
     |████████████████████████████████| 286kB 12.4MB/s 
     |████████████████████████████████| 776.8MB 20kB/s 
     |████████████████████████████████| 798kB 61.0MB/s 
     |████████████████████████████████| 1.2MB 57.1MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 2.1MB 60.0MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 983kB 56.8MB/s 
     |████████████████████████████████| 901kB 64.3MB/s 
     |████████████████████████████████| 3.3MB 50.1MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=cbc2498e7b648bf9f8060c5f1aa9b0e2763d1f0cd3c03daecaa1104bcdf0e95d
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc

100%|██████████| 265512723/265512723 [00:06<00:00, 39131690.68B/s]

2021-04-27 21:37:41,248 copying /tmp/tmp5pbtetp4 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-04-27 21:37:41,629 removing temp file /tmp/tmp5pbtetp4
2021-04-27 21:37:43,744 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def discretize_prices(n):
  if n > 0:
    return 1
  else:
    return 0
def stock_process(ticker):
  json_list = []
  clean_json_list = []
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  labels = prices['Date']
  prices = prices.drop(['Date'], axis = 1)
  prices = prices.diff(periods=-1) * -1
  prices['Date'] = labels
  prices = prices.drop(['Open', 'High','Low', 'Adj Close', 'Volume'], axis = 1)
  prices.drop(prices.tail(1).index,inplace=True)
  prices['Close'] = prices['Close'].apply(discretize_prices)
  count = 0
  N = len(prices)
  sp = spacy.load('en_core_web_sm')
  all_stopwords = sp.Defaults.stop_words
  lemmatizer = WordNetLemmatizer()
  for i in range(N):
    date = prices.Date.iloc[i]
    file_path = 'tweet/preprocessed/' + ticker + '/' + date
    if os.path.isfile(file_path):
      tweets = open(file_path)
      for line in tweets:
        json_val = json.loads(line)
        json_val['text'] = list(filter(('AT_USER').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('URL').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('rt').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('?').__ne__, json_val.get('text')))
        json_val['text'] = list(filter((ticker).__ne__, json_val.get('text')))
        json_val['text'] = list(filter((ticker.lower()).__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('\u2026').__ne__, json_val.get('text')))
        text = ' '.join(json_val['text'])
        removetable = str.maketrans('', '', string.punctuation)
        text = text.translate(removetable)
        text_tokens = word_tokenize(text)
        tokens_without_sw= [lemmatizer.lemmatize(word) for word in text_tokens if not word in all_stopwords]
        if len(tokens_without_sw) >= 5:
          flair_sentence = flair.data.Sentence(' '.join(tokens_without_sw))
          sentiment_model.predict(flair_sentence)
          probability = flair_sentence.labels[0].value
          assurance = flair_sentence.labels[0].score
          if probability == 'POSITIVE':
            probability = 1
          else:
            probability = 0          
          json_object = {'text': tokens_without_sw ,
                          'label': int(prices.Close.iloc[i])
                        }
          if (probability == prices.Close.iloc[i] and assurance > 0.75):
            clean_json_list.append(json_object)
          else: 
            json_list.append(json_object)
      tweets.close()
  return json_list, clean_json_list
def pos_stock_process(ticker):
  json_list = []
  clean_json_list = []
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  labels = prices['Date']
  prices = prices.drop(['Date'], axis = 1)
  prices = prices.diff(periods=-1) * -1
  prices['Date'] = labels
  prices = prices.drop(['Open', 'High','Low', 'Adj Close', 'Volume'], axis = 1)
  prices.drop(prices.tail(1).index,inplace=True)
  prices['Close'] = prices['Close'].apply(discretize_prices)
  count = 0
  N = len(prices)
  sp = spacy.load('en_core_web_sm')
  all_stopwords = sp.Defaults.stop_words
  lemmatizer = WordNetLemmatizer()
  for i in range(N):
    date = prices.Date.iloc[i]
    file_path = 'tweet/preprocessed/' + ticker + '/' + date
    if os.path.isfile(file_path):
      tweets = open(file_path)
      for line in tweets:
        json_val = json.loads(line)
        json_val['text'] = list(filter(('AT_USER').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('URL').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('rt').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('?').__ne__, json_val.get('text')))
        json_val['text'] = list(filter((ticker).__ne__, json_val.get('text')))
        json_val['text'] = list(filter((ticker.lower()).__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('\u2026').__ne__, json_val.get('text')))
        text = ' '.join(json_val['text'])
        removetable = str.maketrans('', '', string.punctuation)
        text = text.translate(removetable)
        text_tokens = word_tokenize(text)
        tokens_without_sw= [lemmatizer.lemmatize(word) for word in text_tokens if not word in all_stopwords]
        if len(tokens_without_sw) >= 5:
          flair_sentence = flair.data.Sentence(' '.join(tokens_without_sw))
          sentiment_model.predict(flair_sentence)
          probability = flair_sentence.labels[0].value
          assurance = flair_sentence.labels[0].score
          if probability == 'POSITIVE':
            probability = 1
          else:
            probability = 0          
          json_object = {'text': tokens_without_sw ,
                          'label': int(prices.Close.iloc[i])
                        }
          if (probability == prices.Close.iloc[i] and assurance > 0.75 and (probability == 1 or (probability == 0 and len(tokens_without_sw) % 6 == 0))):
            clean_json_list.append(json_object)
          else: 
            json_list.append(json_object)
      tweets.close()
  return json_list, clean_json_list
def write_to_files(json_data, percent, clean = True, meta_set = None, combined = False):
  if meta_set == None:
    fraction = percent / 100
    copy = json_data
    random.shuffle(copy)
    copy = copy[0:int(len(copy) * fraction)]
    num_train = int(len(copy) * .7)
    train_set = copy[0:num_train]
    test_set = copy[(num_train):(len(copy)-1)]
    if combined:
      if clean:
        train_name = 'combined_train_set' + str(percent) + ".json"
        test_name = 'combined_train_set'+ str(percent) +".json" 
      else:
        train_name = 'combined_messy_train_set_' + str(percent) + ".json"
        test_name = 'combined_messy_test_set_'+ str(percent) +".json" 
    else:
      if clean:
        train_name = 'train_set' + str(percent) + ".json"
        test_name = 'train_set'+ str(percent) +".json" 
      else:
        train_name = 'messy_train_set_' + str(percent) + ".json"
        test_name = 'messy_test_set_'+ str(percent) +".json" 
    with open(train_name, 'w+') as openfile:
      openfile.write('\n'.join(json.dumps(i) for i in train_set))
    with open(test_name, 'w+') as openfile:
      openfile.write('\n'.join(json.dumps(i) for i in test_set))
  else:
    fraction = percent / 100
    copy = json_data
    meta_copy = meta_set
    random.shuffle(copy)
    random.shuffle(meta_copy)
    copy = copy[0:int(len(copy) * fraction)]
    meta_copy = meta_copy[0:int(len(meta_copy) * fraction)]
    num_train = int(len(copy) * .7)
    num_meta_train = int(len(meta_copy) * .7)
    train_set = copy[0:num_train]
    test_set = copy[(num_train):(len(copy)-1)]
    train_set = train_set + meta_copy[0:num_meta_train]
    random.shuffle(train_set)
    test_set = test_set + meta_copy[(num_meta_train+1):(len(meta_copy)-1)]
    random.shuffle(test_set)
    meta_test_copy = meta_copy[(num_meta_train+1):(len(meta_copy)-1)]
    meta_copy = meta_copy[0:num_meta_train]
    if combined:
      if clean == True:
        train_name = 'posclean_train_set_' + str(percent) + ".json"
        test_name = 'posclean_test_set_'+ str(percent) +".json" 
        meta_name = 'posclean_meta_set_' + str(percent) + '.json'
        meta_test = 'posclean_meta_test_set' + str(percent) + '.json'
      else:
        train_name = 'combined_messy_train_set_' + str(percent) + ".json"
        test_name = 'combined_messy_test_set_'+ str(percent) +".json" 
        meta_name = 'combined_messy_meta_set_' + str(percent) + '.json'
        meta_test = 'combined_messy_meta_test_set' + str(percent) + '.json'
    else:
      if clean == True:
        train_name = 'train_set_' + str(percent) + ".json"
        test_name = 'test_set_'+ str(percent) +".json" 
        meta_name = 'meta_set_' + str(percent) + '.json'
        meta_test = 'meta_test_set_' + str(percent) + '.json'
      else:
        train_name = 'messy_train_set_' + str(percent) + ".json"
        test_name = 'messy_test_set_'+ str(percent) +".json" 
        meta_name = 'messy_meta_set_' + str(percent) + '.json'
        meta_test = 'messy_meta_test_set' + str(percent) + '.json'
    with open(train_name, 'w+') as openfile:
      openfile.write('\n'.join(json.dumps(i) for i in train_set))
    with open(test_name, 'w+') as openfile:
      openfile.write('\n'.join(json.dumps(i) for i in test_set))
    with open(meta_name, 'w+') as openfile:
      openfile.write('\n'.join(json.dumps(i) for i in meta_copy))
    with open(meta_test, 'w+') as openfile:
      openfile.write('\n'.join(json.dumps(i) for i in meta_test_copy))
def write_aapl():
  processed = stock_process('AAPL')
  write_to_files(processed, 100)
def write_clean_to_files():
  tickers = os.listdir("tweet/preprocessed")
  #tickers = ['AAPL']
  end_json_vals = []
  end_clean_vals = []
  for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    print("Current Ticker: ", ticker)
    processed, clean_processed = combined_stock_process(ticker)
    if processed:
      end_json_vals = end_json_vals + processed
      end_clean_vals = end_clean_vals + clean_processed
      print("Success! There are now ", len(end_json_vals), " points", "and ", len(end_clean_vals), " confirmed clean data points")
  write_to_files(end_json_vals, 50, meta_set = end_clean_vals, combined = True)

def write_posclean_to_files():
  tickers = os.listdir("tweet/preprocessed")
  #tickers = ['AAPL']
  end_json_vals = []
  end_clean_vals = []
  for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    print("Current Ticker: ", ticker)
    processed, clean_processed = pos_stock_process(ticker)
    if processed:
      end_json_vals = end_json_vals + processed
      end_clean_vals = end_clean_vals + clean_processed
      print("Success! There are now ", len(end_json_vals), " points", "and ", len(end_clean_vals), " confirmed clean data points")
  write_to_files(end_json_vals, 100, meta_set = end_clean_vals, combined = True)

def write_messy_to_files():
  tickers = os.listdir("tweet/preprocessed")
  #tickers = ['AAPL']
  end_json_vals = []
  end_clean_vals = []
  for i in tqdm(range(len(tickers))):
    ticker = tickers[i]
    print("Current Ticker: ", ticker)
    processed, clean_processed = combined_messy_stock_process(ticker)
    if processed:
      end_json_vals = end_json_vals + processed
      end_clean_vals = end_clean_vals + clean_processed
      print("Success! There are now ", len(end_json_vals), " points", "and ", len(end_clean_vals), " confirmed clean data points")
  write_to_files(end_json_vals, 50, meta_set = end_clean_vals, combined = True, clean = False)
def combined_stock_process(ticker):
  json_list = []
  clean_json_list = []
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  labels = prices['Date']
  prices = prices.drop(['Date'], axis = 1)
  prices = prices.diff(periods=-1) * -1
  prices['Date'] = labels
  prices = prices.drop(['Open', 'High','Low', 'Adj Close', 'Volume'], axis = 1)
  prices.drop(prices.tail(1).index,inplace=True)
  prices['Close'] = prices['Close'].apply(discretize_prices)
  count = 0
  N = len(prices)
  sp = spacy.load('en_core_web_sm')
  all_stopwords = sp.Defaults.stop_words
  lemmatizer = WordNetLemmatizer()
  for i in range(N):
    day_tweets = []
    date = prices.Date.iloc[i]
    file_path = 'tweet/preprocessed/' + ticker + '/' + date
    if os.path.isfile(file_path):
      tweets = open(file_path)
      for line in tweets:
        json_val = json.loads(line)
        json_val['text'] = list(filter(('AT_USER').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('URL').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('rt').__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('?').__ne__, json_val.get('text')))
        json_val['text'] = list(filter((ticker).__ne__, json_val.get('text')))
        json_val['text'] = list(filter((ticker.lower()).__ne__, json_val.get('text')))
        json_val['text'] = list(filter(('\u2026').__ne__, json_val.get('text')))
        text = ' '.join(json_val['text'])
        removetable = str.maketrans('', '', string.punctuation)
        text = text.translate(removetable)
        text_tokens = word_tokenize(text)
        tokens_without_sw= [lemmatizer.lemmatize(word) for word in text_tokens if not word in all_stopwords]
        day_tweets = day_tweets + tokens_without_sw
      tweets.close()
      if len(tokens_without_sw) >= 5:
        flair_sentence = flair.data.Sentence(' '.join(tokens_without_sw))
        sentiment_model.predict(flair_sentence)
        probability = flair_sentence.labels[0].value
        assurance = flair_sentence.labels[0].score
        if probability == 'POSITIVE':
          probability = 1
        else:
          probability = 0          
        json_object = {'text': tokens_without_sw ,
                        'label': int(prices.Close.iloc[i])
                      }
        if (probability == prices.Close.iloc[i] and assurance > 0.75):
          clean_json_list.append(json_object)
        else: 
          json_list.append(json_object)
  return json_list, clean_json_list
def combined_messy_stock_process(ticker):
  json_list = []
  clean_json_list = []
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  prices = pd.read_csv('price/raw/'+ticker+'.csv')
  labels = prices['Date']
  prices = prices.drop(['Date'], axis = 1)
  prices = prices.diff(periods=-1) * -1
  prices['Date'] = labels
  prices = prices.drop(['Open', 'High','Low', 'Adj Close', 'Volume'], axis = 1)
  prices.drop(prices.tail(1).index,inplace=True)
  prices['Close'] = prices['Close'].apply(discretize_prices)
  count = 0
  N = len(prices)
  sp = spacy.load('en_core_web_sm')
  all_stopwords = sp.Defaults.stop_words
  lemmatizer = WordNetLemmatizer()
  for i in range(N):
    day_tweets = []
    date = prices.Date.iloc[i]
    file_path = 'tweet/preprocessed/' + ticker + '/' + date
    if os.path.isfile(file_path):
      tweets = open(file_path)
      for line in tweets:
        json_val = json.loads(line)
        text = ' '.join(json_val['text'])
        text_tokens = word_tokenize(text)
        day_tweets = day_tweets + text_tokens
      tweets.close()
    if len(day_tweets) >= 5:
      flair_sentence = flair.data.Sentence(' '.join(day_tweets))
      sentiment_model.predict(flair_sentence)
      probability = flair_sentence.labels[0].value
      assurance = flair_sentence.labels[0].score
      if probability == 'POSITIVE':
        probability = 1
      else:
        probability = 0          
      json_object = {'text': day_tweets,
                      'label': int(prices.Close.iloc[i])
                    }
      if (probability == prices.Close.iloc[i] and assurance > 0.75):
        clean_json_list.append(json_object)
      else: 
        json_list.append(json_object)
  return json_list, clean_json_list

In [ ]:
write_clean_to_files()

In [ ]:
write_messy_to_files()

#Twitter API Calls

In [ ]:
relevantVals = []
num_pos = 0
num_neg = 0
with open('model_data/combined_meta_set_100.json') as openfile:
  for line in openfile:
    json_val = json.loads(line)
    if json_val['label'] == 1:
      relevantVals.append(json_val)
      num_neg = num_neg + 1
    else:
      if (num_pos < num_neg / 4):
        relevantVals.append(json_val)
        num_pos += 1
print(f"num_pos: {num_pos}, num_neg: {num_neg}")
#with open('modifiedPos_meta_set_100.json', 'w+') as openfile:
#      openfile.write('\n'.join(json.dumps(i) for i in relevantVals))
num_pos = 0
num_neg = 0
with open('model_data/combined_messy_meta_set_100.json') as openfile:
  for line in openfile:
    json_val = json.loads(line)
    if json_val['label'] == 0:
      num_neg += 1
    else:
      num_pos += 1
print(num_pos)
print(num_neg)

num_pos: 621, num_neg: 2484
978
4947


In [ ]:
write_posclean_to_files()